In [ ]:
!pip install gensim==4.1.2

     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/data.zip

Archive:  /content/drive/MyDrive/data.zip
  inflating: data/testing_corpus.csv  
  inflating: data/testing_queries.csv  
  inflating: data/testing_result.csv  
  inflating: data/training_corpus.csv  
  inflating: data/training_queries.csv  
  inflating: data/training_result.csv  


In [ ]:
import pandas as pd
import numpy as np
import spacy

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
nlp.max_length=5000000

In [ ]:
cd data

/content/data


In [ ]:
training_corpus=pd.read_csv("training_corpus.csv")

In [ ]:
testing_corpus=pd.read_csv("testing_corpus.csv")

In [ ]:
training_queries=pd.read_csv("training_queries.csv")

In [ ]:
testing_queries=pd.read_csv("testing_queries.csv")

In [ ]:
training_queries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     1000 non-null   int64 
 1   query   1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
testing_queries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     1000 non-null   int64 
 1   query   1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
import re

# Lowercasing the text
training_corpus['cleaned']=training_corpus['body'].apply(lambda x:x.lower())
testing_corpus['cleaned']=testing_corpus['body'].apply(lambda x:x.lower())

# Dictionary of english Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)


In [ ]:
def clean_text(text):
    text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z]',' ',text)
    return text


In [ ]:
# Lowercasing the text
training_queries['cleaned']=training_queries['query'].apply(lambda x:x.lower())
testing_queries['cleaned']=testing_queries['query'].apply(lambda x:x.lower())

# Expanding contractions
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x:expand_contractions(x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x:expand_contractions(x))

# Cleaning queries using RegEx
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x: clean_text(x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x: clean_text(x))

# Removing extra spaces
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x: re.sub(' +',' ',x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x: re.sub(' +',' ',x))

In [ ]:
# Combining corpus and queries for training
combined_training=pd.concat([training_corpus.rename(columns={'lemmatized':'text'})['text'],training_queries.rename(columns={'cleaned':'text'})['text']]).sample(frac=1).reset_index(drop=True)

In [ ]:
pip install sentence_transformers

     |████████████████████████████████| 78 kB 3.0 MB/s 
     |████████████████████████████████| 3.4 MB 10.9 MB/s 
     |████████████████████████████████| 6.8 MB 37.4 MB/s 
     |████████████████████████████████| 1.2 MB 50.3 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 26.2 MB/s 
     |████████████████████████████████| 895 kB 62.7 MB/s 
     |████████████████████████████████| 3.3 MB 19.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=9b027cedaf1a1629f2fe1cd437669e718bfcf66d75a5643534d9b16af491a2e6
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(768)
    else:
        embeddings.append(sbert_model.encode(doc_tokens))
        #print("done")
        #return np.array(embeddings)
        return np.mean(embeddings, axis=0)
# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x))

cpy_corpus=testing_corpus['vector']
cpy_queries=testing_queries['vector']

In [ ]:
for i in range(len(cpy_corpus)):
    if cpy_corpus[i].shape[0] == 1536:
        print(True)

In [ ]:
(testing_result=pd.read_csv("testing_result.csv")
training_result=pd.read_csv("training_result.csv")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 0.7396544979843791


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  vector=get_embedding_w2v(query)
  #vector = pca.transform(d)
  # ranking documents
  print(vector.shape)
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
ranking_ir('michael jordan')

(768,)


,docid,title,body,similarity,euclidean_distances
0,D1694409,QARMmeansQuality Assurance Requirement Matrix,QARMmeans Quality Assurance Requirement Matrix...,0.465945,16.860855
1,D330015,Chelsea W.Size: 554bones in the axial skeleton...,Chelsea W. Size: 554bones in the axial skeleto...,0.420865,17.003351
2,D79684,Wedding Cost Locations,"(00000) United States (01001) Agawam, MA (010...",0.416837,17.449272
3,D834407,.,Dilemmes & doutes« D & D »Sujet:rectangleliste...,0.409064,17.375959
4,D2684579,.,One entry found for fruition. Main Entry: fru·...,0.407469,17.855337
5,D2138629,Intizar Ma Kawa Lyrics â Karan Khan,Intizar Ma Kawa Lyrics – Karan Khanby Waqas Si...,0.399002,17.251917
6,D2443747,What is T24,What is T24T24 is the core banking system deve...,0.390369,17.905886
7,D1336656,orthocenter,Contents1 English1.1 Alternative forms1.2 Etym...,0.387725,17.802851
8,D1045584,Head Start Program Search,Cities in North Carolina A Aberdeen (2)Ahoskie...,0.382118,17.535048
9,D3381366,410 Area Code,Area code 410 is an area code that covers East...,0.379856,17.768604


In [ ]:
ranking_ir('nba championship michael jordan')

(768,)


,docid,title,body,similarity,euclidean_distances
0,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.791837,10.628949
1,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.781661,10.864708
2,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.773679,11.001353
3,D2688388,How many career points did Michael Jordan score?,Yaaaaa23 2 Contributions How many career point...,0.765721,11.203687
4,D3376585,6 reasons LeBron James will be better than Mic...,6 reasons Le Bron James will be better than Mi...,0.758931,11.353348
5,D1249158,The NBAâs 13 Most Successful Players of All Time,The Larry O’Brien trophy is raised. | Andy Lyo...,0.746410,11.646491
6,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.743929,11.762372
7,D2733703,How Many Championships Does Kobe Have?,Sports & Active Lifestyle Athletes Q: How Many...,0.743253,11.814071
8,D2806221,How much money do NBA players make in a year?,Answers.com ® Wiki Answers ® Categories Sports...,0.739609,11.847598
9,D1169785,How many MVP's did Kobe Bryant get?,Answers.com ® Wiki Answers ® Categories Entert...,0.737589,11.815479


In [ ]:
# Function returning vector reperesentation of a document
'''def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(768)
    else:
        embeddings.append(sbert_model.encode(doc_tokens))
        print("done")
        #return np.array(embeddings)
        return np.mean(embeddings, axis=0)
# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x))'''
testing_corpus['vector']=cpy_corpus
testing_queries['vector']=cpy_queries

In [ ]:
a = np.zeros((testing_corpus['vector'].shape[0],testing_corpus['vector'][0].shape[0]))
for i in range(len(testing_corpus['vector'])):
    a[i,:] = testing_corpus['vector'][i]

b = np.zeros((testing_queries['vector'].shape[0],testing_queries['vector'][0].shape[0]))
for i in range(len(testing_queries['vector'])):
    b[i,:] = testing_queries['vector'][i]

from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis

pca1 = PCA(n_components=768)
aa = pca1.fit_transform(a)
#pca1 = PCA(n_components = 300)
bb = pca1.transform(b)

pca = FactorAnalysis(n_components=750)
testing_corpus1 = pca.fit_transform(aa)
#pca1 = PCA(n_components = 300)
testing_queries1 = pca.transform(bb)

In [ ]:
for i in range(len(testing_corpus['vector'])):
     testing_corpus['vector'][i]=testing_corpus1[i]
for i in range(len(testing_queries['vector'])):
     testing_queries['vector'][i]=testing_queries1[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 0.7043219148400095


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  d=get_embedding_w2v(query).reshape(1, -1)
  d=pca1.transform(d)
  vector = pca.transform(d)
  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
ranking_ir('michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D2305993,.,Adams.......... (Area Code 541).....566 Adel.....,0.793660,3.349149
1,D834407,.,Dilemmes & doutes« D & D »Sujet:rectangleliste...,0.790202,3.342491
2,D330015,Chelsea W.Size: 554bones in the axial skeleton...,Chelsea W. Size: 554bones in the axial skeleto...,0.769802,3.459094
3,D79684,Wedding Cost Locations,"(00000) United States (01001) Agawam, MA (010...",0.756235,3.547801
4,D2148689,What is a word that can be a verb a noun adjec...,"Notyouraveragedummy ... 327,032 Contributions ...",0.744634,3.627479
5,D2394176,What is the supreme law of the land in the Uni...,"Ma9989009 464,381 Contributions What is the su...",0.730490,3.706326
6,D2138629,Intizar Ma Kawa Lyrics â Karan Khan,Intizar Ma Kawa Lyrics – Karan Khanby Waqas Si...,0.726576,3.724127
7,D2598248,"Chapter 30: Blood vessels, lymphatic drainage ...","On this page Abdominal aorta (figs. 13-5, 30-1...",0.718119,3.772731
8,D1045584,Head Start Program Search,Cities in North Carolina A Aberdeen (2)Ahoskie...,0.715059,3.788974
9,D1463506,THE ORIGIN OF SOME ENGLISH PLACE NAMES,"BURY, BOROUGHThis is usually a corruption of b...",0.712130,3.920860


In [ ]:
ranking_ir('nba championship michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.896623,2.252899
1,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.895053,2.293396
2,D1169785,How many MVP's did Kobe Bryant get?,Answers.com ® Wiki Answers ® Categories Entert...,0.866421,2.549138
3,D2806221,How much money do NBA players make in a year?,Answers.com ® Wiki Answers ® Categories Sports...,0.863163,2.569644
4,D2688388,How many career points did Michael Jordan score?,Yaaaaa23 2 Contributions How many career point...,0.854291,2.652741
5,D3376585,6 reasons LeBron James will be better than Mic...,6 reasons Le Bron James will be better than Mi...,0.849794,2.685431
6,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.848168,2.695282
7,D1415956,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.847843,2.698876
8,D2986655,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.847843,2.698876
9,D2733703,How Many Championships Does Kobe Have?,Sports & Active Lifestyle Athletes Q: How Many...,0.839381,2.779661


In [ ]:
# Function returning vector reperesentation of a document
'''def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(768)
    else:
        embeddings.append(sbert_model.encode(doc_tokens))
        print("done")
        #return np.array(embeddings)
        return np.mean(embeddings, axis=0)
# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x))'''
testing_corpus['vector']=cpy_corpus
testing_queries['vector']=cpy_queries

In [ ]:
a = np.zeros((testing_corpus['vector'].shape[0],testing_corpus['vector'][0].shape[0]))
for i in range(len(testing_corpus['vector'])):
    a[i,:] = testing_corpus['vector'][i]

b = np.zeros((testing_queries['vector'].shape[0],testing_queries['vector'][0].shape[0]))
for i in range(len(testing_queries['vector'])):
    b[i,:] = testing_queries['vector'][i]

from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis

pca1 = FactorAnalysis(n_components=750)
aa = pca1.fit_transform(a)
#pca1 = PCA(n_components = 300)
bb = pca1.transform(b)

pca = PCA(n_components=750)
testing_corpus1 = pca.fit_transform(aa)
#pca1 = PCA(n_components = 300)
testing_queries1 = pca.transform(bb)

In [ ]:
for i in range(len(testing_corpus['vector'])):
     testing_corpus['vector'][i]=testing_corpus1[i]
for i in range(len(testing_queries['vector'])):
     testing_queries['vector'][i]=testing_queries1[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 0.7522140989543958


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  d=get_embedding_w2v(query).reshape(1, -1)
  d=pca1.transform(d)
  vector = pca.transform(d)
  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
ranking_ir('michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D2072045,.,Error. Page cannot be displayed. Please contac...,0.314050,72.553438
1,D2443747,What is T24,What is T24T24 is the core banking system deve...,0.233081,73.095354
2,D2684579,.,One entry found for fruition. Main Entry: fru·...,0.221953,74.046984
3,D1058998,Celebrities With Two Different Color Eyes,Previous Next Image 1 of 8 Toggle Two Colors H...,0.220757,73.857643
4,D3175415,.,Oops! There was a problem!There was an unexpec...,0.214143,73.123220
5,D1694409,QARMmeansQuality Assurance Requirement Matrix,QARMmeans Quality Assurance Requirement Matrix...,0.203617,73.154793
6,D1102219,"Prince Andrew, his daughters and their hats ar...","Lifestyle Prince Andrew, his daughters and the...",0.201195,76.220380
7,D1967392,Rewind the Texas action,"An unanticipated problem was encountered, chec...",0.190307,72.391762
8,D761399,Vito's in the Valley,Vito's in the Valley76 Reviews #53 of 158 Rest...,0.184144,73.714625
9,D1561743,.,Username: New User? Locate ATM or Branch Rate5...,0.158761,76.352335


In [ ]:
ranking_ir('nba championship michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.470716,51.609881
1,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.424715,53.098459
2,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.361509,54.916797
3,D3376585,6 reasons LeBron James will be better than Mic...,6 reasons Le Bron James will be better than Mi...,0.351616,55.007979
4,D1249158,The NBAâs 13 Most Successful Players of All Time,The Larry O’Brien trophy is raised. | Andy Lyo...,0.347615,55.043119
5,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.330009,55.283310
6,D2688388,How many career points did Michael Jordan score?,Yaaaaa23 2 Contributions How many career point...,0.328971,56.097109
7,D2796474,How many championships does Michael Jordan have?,Jim Schiller 1 Contribution How many champions...,0.301380,57.201902
8,D2892646,How many championship rings does Michael Jorda...,How many championship rings does Michael Jorda...,0.297024,57.230628
9,D2733703,How Many Championships Does Kobe Have?,Sports & Active Lifestyle Athletes Q: How Many...,0.296709,56.388898


In [ ]:
# Function returning vector reperesentation of a document
'''def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(768)
    else:
        embeddings.append(sbert_model.encode(doc_tokens))
        print("done")
        #return np.array(embeddings)
        return np.mean(embeddings, axis=0)
# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x))'''

In [ ]:
testing_corpus['vector']=cpy_corpus
testing_queries['vector']=cpy_queries

In [ ]:
a = np.zeros((testing_corpus['vector'].shape[0],testing_corpus['vector'][0].shape[0]))
for i in range(len(testing_corpus['vector'])):
    a[i,:] = testing_corpus['vector'][i]

b = np.zeros((testing_queries['vector'].shape[0],testing_queries['vector'][0].shape[0]))
for i in range(len(testing_queries['vector'])):
    b[i,:] = testing_queries['vector'][i]

from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis

pca1 = FactorAnalysis(n_components=750)
aa = pca1.fit_transform(a)
#pca1 = PCA(n_components = 300)
bb = pca1.transform(b)

pca = FactorAnalysis(n_components=750)
testing_corpus1 = pca.fit_transform(aa)
#pca1 = PCA(n_components = 300)
testing_queries1 = pca.transform(bb)

In [ ]:
for i in range(len(testing_corpus['vector'])):
     testing_corpus['vector'][i]=testing_corpus1[i]
for i in range(len(testing_queries['vector'])):
     testing_queries['vector'][i]=testing_queries1[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 0.6158216400541698


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  d=get_embedding_w2v(query).reshape(1, -1)
  d=pca1.transform(d)
  vector = pca.transform(d)
  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
ranking_ir('michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D1014538,City Clerk,City Clerk City Commissions Petaluma encourage...,0.947323,0.426101
1,D1210881,Chapter 7- Section 3- Protists,22 terms pieman123456Chapter 7- Section 3- Pro...,0.946447,0.564993
2,D2663389,Welcome,Announcements Most Requested Pages Public Meet...,0.942648,0.530793
3,D3087036,Welcome,Announcements Most Requested Pages Public Meet...,0.942648,0.530793
4,D1862438,What was the worst concentration camp or death...,Answers.com ® Wiki Answers ® Categories Histor...,0.940501,0.404960
5,D288023,Maunder Minimum,"From Wikipedia, the free encyclopedianavigatio...",0.938293,0.386634
6,D1862792,Sugar refinery,"From Wikipedia, the free encyclopedianavigatio...",0.935175,0.521273
7,D373682,United States Court of Appeals for the Federal...,"From Wikipedia, the free encyclopedia (Redirec...",0.934715,0.504123
8,D1726194,FAQ,FAQHead Lice Frequently Asked Questions Where ...,0.933812,0.518396
9,D3058321,Which rivers in the Northern Hemisphere flow f...,Northern Hemisphere Rivers Science of Everyday...,0.933582,0.479156


In [ ]:
ranking_ir('nba championship michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D3323744,HIPAA TRAINING REQUIREMENTS: FAQ,HIPAA TRAINING REQUIREMENTS: FAQby Daniel J. S...,0.940959,0.738058
1,D2044454,Hudson,Hudson Rating : Excellent0.5 1 1.5 2 2.5 3 3.5...,0.922223,0.811493
2,D985990,Masturbation,Masturbation Intro Symptoms Causes Tests Treat...,0.918466,0.905874
3,D3214444,DRYWALL TEXTURED PAINT,DRYWALL TEXTURED PAINTThere are a number of wa...,0.911489,0.876928
4,D2082795,Top 10 Foods to Eat During Cold and Flu,142 votesvote!Top 10 Foods to Eat During Cold ...,0.909969,0.761177
5,D3042692,The High School Math Courses You Should Take,The High School Math Courses You Should Take P...,0.906077,0.775646
6,D2860608,What components make up the respiratory system?,What components make up the respiratory system...,0.905750,0.774317
7,D1746337,What Version of Windows Do I Have?,How To ›Windows What Version of Windows Do I H...,0.900365,0.809678
8,D1711545,What Version of Windows Do I Have?,How To ›Windows What Version of Windows Do I H...,0.900365,0.809678
9,D1316727,What organization is the Taco Bell foundation ...,Answers.com ® Wiki Answers ® Categories Food &...,0.900258,0.695666


In [ ]:
# Function returning vector reperesentation of a document
'''def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(768)
    else:
        embeddings.append(sbert_model.encode(doc_tokens))
        print("done")
        #return np.array(embeddings)
        return np.mean(embeddings, axis=0)
# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x))'''
testing_corpus['vector']=cpy_corpus
testing_queries['vector']=cpy_queries

In [ ]:
a = np.zeros((testing_corpus['vector'].shape[0],testing_corpus['vector'][0].shape[0]))
for i in range(len(testing_corpus['vector'])):
    a[i,:] = testing_corpus['vector'][i]

b = np.zeros((testing_queries['vector'].shape[0],testing_queries['vector'][0].shape[0]))
for i in range(len(testing_queries['vector'])):
    b[i,:] = testing_queries['vector'][i]

from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis

pca = FactorAnalysis(n_components=750)
testing_corpus1 = pca.fit_transform(a)
#pca1 = PCA(n_components = 300)
testing_queries1 = pca.transform(b)

In [ ]:
for i in range(len(testing_corpus['vector'])):
     testing_corpus['vector'][i]=testing_corpus1[i]
for i in range(len(testing_queries['vector'])):
     testing_queries['vector'][i]=testing_queries1[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 0.7519974322877291


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  d=get_embedding_w2v(query).reshape(1, -1)
  #d=pca1.transform(d)
  vector = pca.transform(d)
  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
bm=ranking_ir('michael jordan')
bm

,docid,title,body,similarity,euclidean_distances
0,D2072045,.,Error. Page cannot be displayed. Please contac...,0.314050,72.553438
1,D2443747,What is T24,What is T24T24 is the core banking system deve...,0.233081,73.095354
2,D2684579,.,One entry found for fruition. Main Entry: fru·...,0.221953,74.046984
3,D1058998,Celebrities With Two Different Color Eyes,Previous Next Image 1 of 8 Toggle Two Colors H...,0.220757,73.857643
4,D3175415,.,Oops! There was a problem!There was an unexpec...,0.214143,73.123220
5,D1694409,QARMmeansQuality Assurance Requirement Matrix,QARMmeans Quality Assurance Requirement Matrix...,0.203617,73.154793
6,D1102219,"Prince Andrew, his daughters and their hats ar...","Lifestyle Prince Andrew, his daughters and the...",0.201195,76.220380
7,D1967392,Rewind the Texas action,"An unanticipated problem was encountered, chec...",0.190307,72.391762
8,D761399,Vito's in the Valley,Vito's in the Valley76 Reviews #53 of 158 Rest...,0.184144,73.714625
9,D1561743,.,Username: New User? Locate ATM or Branch Rate5...,0.158761,76.352335


In [ ]:
ranking_ir('nba championship michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.470716,51.609881
1,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.424715,53.098459
2,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.361509,54.916797
3,D3376585,6 reasons LeBron James will be better than Mic...,6 reasons Le Bron James will be better than Mi...,0.351616,55.007979
4,D1249158,The NBAâs 13 Most Successful Players of All Time,The Larry O’Brien trophy is raised. | Andy Lyo...,0.347615,55.043119
5,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.330009,55.283310
6,D2688388,How many career points did Michael Jordan score?,Yaaaaa23 2 Contributions How many career point...,0.328971,56.097109
7,D2796474,How many championships does Michael Jordan have?,Jim Schiller 1 Contribution How many champions...,0.301380,57.201902
8,D2892646,How many championship rings does Michael Jorda...,How many championship rings does Michael Jorda...,0.297024,57.230628
9,D2733703,How Many Championships Does Kobe Have?,Sports & Active Lifestyle Athletes Q: How Many...,0.296709,56.388898
